In [1]:
# Required imports
import tkinter as tk
from tkinter import ttk
from tkinter import StringVar, Listbox, MULTIPLE
import requests
import pandas as pd
from datetime import datetime

# List of known bookmakers
known_bookmakers = ['FanDuel', 'BetUS', 'DraftKings', 'SuperBook', 'Caesars', 'LowVig.ag', 'BetOnline.ag', 'Bovada', 'TwinSpires', 'BetRivers', 'Unibet', 'MyBookie.ag', 'PointsBet (US)', 'WynnBET', 'BetMGM', 'Barstool Sportsbook']

# Initialize global variables for API quota
api_requests_used = 0
api_requests_remaining = 0

# Fetch and prepare odds data
def download_odds(sport, api_quota_label, selected_bookmakers='All'):
    global api_requests_used, api_requests_remaining

    # Replace with your API endpoint and key
    url = f"https://api.the-odds-api.com/v4/sports/{sport}/odds/?apiKey=2c6684b5eb487d4e9e90bfbbffc2d903&regions=us&markets=h2h,spreads&oddsFormat=american"
    response = requests.get(url)
    
    api_requests_used = int(response.headers.get('x-requests-used', '0'))
    api_requests_remaining = int(response.headers.get('x-requests-remaining', '0'))

    # Update the API quota label
    api_quota_label.config(text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")

    result = response.json()
    all_matches = []

    for res in result:
        for books in res['bookmakers']:
            if selected_bookmakers != 'All' and books['title'] not in selected_bookmakers:
                continue
            match = {
                'event_time': datetime.fromisoformat(res['commence_time'].replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M:%S'),
                'bookmaker': books['title'],
                'Fighter': books['markets'][0]['outcomes'][0]['name'],
                'Opponent': books['markets'][0]['outcomes'][1]['name'],
                'odds_f1': books['markets'][0]['outcomes'][0]['price'],
                'odds_f2': books['markets'][0]['outcomes'][1]['price']
            }
            all_matches.append(match)

    return pd.DataFrame(all_matches)

# Function to populate the Tkinter table
def populate_table(tree, df):
    # Clear existing rows
    for row in tree.get_children():
        tree.delete(row)

    # Insert new rows
    for index, row in df.iterrows():
        tree.insert('', 'end', values=(row['event_time'], row['bookmaker'], row['Fighter'], row['odds_f1'], row['Opponent'], row['odds_f2']))

# Function to refresh the data and table
def refresh_table(tree, sport, api_quota_label, bookmaker_listbox):
    selected_indices = bookmaker_listbox.curselection()
    selected_bookmakers = [bookmaker_listbox.get(i) for i in selected_indices]
    
    if "All" in selected_bookmakers:
        selected_bookmakers = 'All'
        
    df = download_odds(sport, api_quota_label, selected_bookmakers)
    populate_table(tree, df)

# Create a mapping from human-readable names to sport keys
sport_name_mapping = {
    'UFC': 'mma_mixed_martial_arts',
    'BOXING': 'boxing_boxing',
    'NFL': 'americanfootball_nfl',
    'NCAAF': 'americanfootball_ncaaf',
    'MLB': 'baseball_mlb',
    'NBA': 'basketball_nba',
    'NHL': 'icehockey_nhl'
}

# Function to create a Treeview for a given sport
def create_sport_tab(tab_control, sport_key, human_readable_name, api_quota_label, auto_refresh_time=0):
    tab = ttk.Frame(tab_control)
    tab_control.add(tab, text=human_readable_name)

    # Create Listbox for bookmakers
    bookmaker_listbox = Listbox(tab, selectmode=MULTIPLE)
    bookmaker_listbox.pack(side='right')

    # Add an "All" option along with known bookmakers
    bookmaker_listbox.insert(tk.END, "All")
    for bm in known_bookmakers:
        bookmaker_listbox.insert(tk.END, bm)

    # Column headers and Treeview
    columns = ('Event Time', 'Bookmaker', 'Player', 'Odds P1', 'Opponent', 'Odds P2')
    tree = ttk.Treeview(tab, columns=columns, show='headings')

    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=100)

    tree.pack(side='left', fill='both', expand=True)
    
    # Scrollbar
    scrollbar = ttk.Scrollbar(tab, orient='vertical', command=tree.yview)
    scrollbar.pack(side='right', fill='y')
    tree.configure(yscrollcommand=scrollbar.set)

    # Modify the button for manual updates
    btn_refresh = ttk.Button(tab, text="Manual Refresh", command=lambda: refresh_table(tree, sport_key, api_quota_label, bookmaker_listbox))
    btn_refresh.pack(side='bottom')

    # Function to refresh the table periodically
    def periodic_refresh():
        refresh_table(tree, sport_key, api_quota_label)
        root.after(auto_refresh_time, periodic_refresh)

    # Start periodic refresh
    if auto_refresh_time:
        periodic_refresh()

# Tkinter GUI setup
root = tk.Tk()
root.title("Live Sports Odds")

# Label to display API quota
api_quota_label = ttk.Label(root, text=f"API Quota: {api_requests_used} used, {api_requests_remaining} remaining")
api_quota_label.pack(side='bottom')

# Create a tab control
tab_control = ttk.Notebook(root)
tab_control.pack(expand=1, fill='both')

# Create tabs dynamically
for human_readable_name, sport_key in sport_name_mapping.items():
    create_sport_tab(tab_control, sport_key, human_readable_name, api_quota_label, auto_refresh_time=0)

# Tkinter event loop
root.mainloop()


: 